# DOMAIN

In [1]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython

from IPython.core import debugger
ist = debugger.set_trace

In [2]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [3]:
import sys
sys.path.insert(0,'../')

## DEFINE

In [4]:
%%run_and_write ../typyPRISM/Space.py
from enum import Enum

class Space(Enum):
    Real    = 1
    Fourier = 2


Overwriting ../typyPRISM/Space.py


In [10]:
%%run_and_write ../typyPRISM/Domain.py
from typyPRISM.Space import Space
import numpy as np
from scipy.fftpack import dst

class Domain:
    '''Define and transform between Real and Fourier space
    
    Domain describes the discretization of Real and Fourier space
    and also sets up the functions and coefficients for transforming
    data between them.
    
    Attributes
    ----------
    length: int
        Number of gridpoints in Real and Fourier space grid
        
    dr,dk: float
        Grid spacing in Real and Fourier space
    
    r,k: float ndarray
        Numpy arrays of grid locations in Real and Fourier space
    
    DST_II_coeffs,DST_III_coeffs: float
        Coefficients needed for Discrete Sine Transforms. Note that these
        values are specific to each implementation of the DST and were 
        derived for (Scipy's interface to) FFTPACK. 
    '''
    __slots__ = ('length',
                 'r','dr',
                 'k','dk',
                 'DST_II_coeffs','DST_III_coeffs')
    def __init__(self,length,dr):
        self.length = length
        
        self.dr = dr
        self.r = np.arange(dr,dr*(length+1),dr)
        
        self.dk = np.pi/(dr*length)
        self.k = np.arange(self.dk,self.dk*(length+1),self.dk)
        
        self.DST_II_coeffs = 2.0*np.pi*self.r*dr 
        self.DST_III_coeffs = self.k * self.dk/(4.0*np.pi*np.pi)
        
    def __repr__(self):
        return '<Domain length:{} dr/rmax:{:4.3f}/{:3.1f} dk/kmax:{:4.3f}/{:3.1f}>'.format(self.length,self.dr,self.r[-1],self.dk,self.k[-1])
    
    def to_fourier(self,array):
        ''' Discrete Sine Transform of a numpy array 
        
        Peforms a Real-to-Real Discrete Sine Transform  of type II 
        on a numpy array of non-complex values. For radial data that is 
        symmetric in \phi and \theta, this is **a** correct transform
        to go from Real-space to Fourier-space. 
        
        Parameters
        ----------
        array: float ndarray
            Real-space data to be transformed
            
        Returns
        -------
        array: float ndarray
            data transformed to fourier space
        
        '''
        return dst(self.DST_II_coeffs*array,type=2)/self.k
    
    def to_real(self,array):
        ''' Discrete Sine Transform of a numpy array 
        
        Peforms a Real-to-Real Discrete Sine Transform  of type III 
        on a numpy array of non-complex values. For radial data that is 
        symmetric in \phi and \theta, this is **a** correct transform
        to go from Fourier-space to Real space.
        
        Parameters
        ----------
        array: float ndarray
            Fourier-space data to be transformed
            
        Returns
        -------
        array: float ndarray
            data transformed to Real space
        
        '''
        return dst(self.DST_III_coeffs*array,type=3)/self.r
    
    def MatrixArray_to_fourier(self,marray):
        ''' Transform all columns of a MatrixArray to Fourier space in-place'''
        if marray.space == Space.Fourier:
            raise ValueError('MatrixArray is marked as already in Fourier space')
            
        for (i,j),column in marray.itercolumn():
            marray[i,j] = self.to_fourier(column)
        
        marray.space = Space.Fourier
            
    def MatrixArray_to_real(self,marray):
        ''' Transform all columns of a MatrixArray to Real space in-place '''
        if marray.space == Space.Real:
            raise ValueError('MatrixArray is marked as already in Real space')
            
        for (i,j),column in marray.itercolumn():
            marray[i,j] = self.to_real(column)
            
        marray.space = Space.Real
            

Overwriting ../typyPRISM/Domain.py


In [13]:
%%run_and_write ../test/Domain_TestCase.py
import unittest
from typyPRISM.MatrixArray import MatrixArray
from typyPRISM.Domain import Domain
import numpy as np

class Domain_TestCase(unittest.TestCase):
    def test_MatrixArray_loop(self):
        '''Can we transform an entire MatrixArray?'''
        length = 1024
        rank = 3
        dr = 0.1
        d = Domain(length=length,dr=dr)
        
        array1 = np.sin(np.arange(0,10*np.pi,0.01))[:length]
        array2 = 5*np.sin(np.arange(0,10*np.pi,0.01))[:length]
        array3 = np.cos(np.arange(0,10*np.pi,0.01))[:length]
        array4 = np.cos(np.arange(0,10*np.pi,0.01))[:length] + np.sin(np.arange(0,10*np.pi,0.01))[:length]
        
        MA = MatrixArray(length=length,rank=rank)
        MA[0,0] = array1
        MA[1,1] = array2
        MA[1,2] = array3
        MA[2,2] = array4
        
        d.MatrixArray_to_fourier(MA)
        np.testing.assert_array_almost_equal(MA[0,0],d.to_fourier(array1))
        np.testing.assert_array_almost_equal(MA[1,1],d.to_fourier(array2))
        np.testing.assert_array_almost_equal(MA[1,2],d.to_fourier(array3))
        np.testing.assert_array_almost_equal(MA[2,1],d.to_fourier(array3))
        np.testing.assert_array_almost_equal(MA[2,2],d.to_fourier(array4))
        
        # This should recover the original values of the MatrixArray
        d.MatrixArray_to_real(MA)
        np.testing.assert_array_almost_equal(MA[0,0],array1)
        np.testing.assert_array_almost_equal(MA[1,1],array2)
        np.testing.assert_array_almost_equal(MA[1,2],array3)
        np.testing.assert_array_almost_equal(MA[2,1],array3)
        np.testing.assert_array_almost_equal(MA[2,2],array4)
        
    def test_array_loop(self):
        '''Can we go to Fourier space and back again?'''
        d = Domain(length=1024,dr=0.1)
        
        real_space_data1 = np.sin(np.arange(0,10*np.pi,0.01))[:1024]
        
        fourier_space_data = d.to_fourier(real_space_data1)
        
        real_space_data2 = d.to_real(fourier_space_data)
        
        # If the DST coefficients are correct, we should get back (within numerical
        # precision) the same array we started with.
        np.testing.assert_array_almost_equal(real_space_data1,real_space_data2)


Overwriting ../typyPRISM/test/Domain_TestCase.py


In [12]:
import unittest
suite = unittest.TestLoader().loadTestsFromTestCase(Domain_TestCase)
unittest.TextTestRunner(verbosity=2).run(suite)

test_MatrixArray_loop (__main__.Domain_TestCase)
Can we transform an entire MatrixArray? ... ok
test_array_loop (__main__.Domain_TestCase)
Can we go to Fourier space and back again? ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.012s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>